This notebook is used to find the corresponding climate and state codes for indvidual waterbodies delineated by 01 res_intersect.ipynb so that they can be diaggregated accordingly.

In [1]:
from functools import reduce
import geopandas as gpd
import fiona
import pickle

In [2]:
fp_clim = 'Documents/GIS DataBase/farmdams/Koppen_Gerrit_Hendriksen_GDA94.shp'
fp_states = 'Documents/GIS DataBase/farmdams/states.shp'
fp_res = 'Documents/GIS DataBase/farmdams/geo_results/res_intersect.shp'

In [3]:
# define function to import selected columns from feature layers for geopandas - similar to usecols for pandas
def records(filename, usecols, **kwargs):
    with fiona.open(filename, **kwargs) as source:
        for feature in source:
            f = {k: feature[k] for k in ['id', 'geometry']}
            f['properties'] = {k: feature['properties'][k] for k in usecols}
            yield f

In [4]:
# import shapefiles
res = gpd.read_file(fp_res)
climate = gpd.GeoDataFrame.from_features(records(fp_clim, usecols = ['FolderPath', 'SymbolID']))
states = gpd.GeoDataFrame.from_features(records(fp_states, usecols = ['STATE_CODE']))

In [5]:
# check if projections match, if not proceed to next cell
print(res.crs == states.crs)
print(res.crs == climate.crs)

False
False


In [8]:
# print crs, if 'None' move to next cell
print(states.crs)
print(climate.crs)

{'init': 'epsg:4283'}
{'init': 'epsg:4283'}


In [7]:
# set crs to res crs
states.crs = {'init': 'epsg:4283'}
climate.crs = {'init': 'epsg:4283'}

In [9]:
# define function to create spatial join from multiple geopandas dataframes - similar to usecols for pandas
def join_reducer(left, right):
    """
    Take two geodataframes, do a spatial join, and return without the
    index_left and index_right columns.
    """
    sjoin = gpd.sjoin(left, right, how='left')
    for column in ['index_right']:
        try:
            sjoin.drop(column, axis=1, inplace=True)
        except ValueError:
            # ignore if there are no index columns
            pass
    return sjoin

In [10]:
# set input frames for spatial join using join_reducer function
input_frames = [res, climate, states]

In [11]:
# spatial join
sjoin = reduce(join_reducer, input_frames)

In [13]:
# save to pickle
with open('Documents/GIS DataBase/farmdams/geo_results/res_sjoin.pickle', 'wb') as handle:
        pickle.dump(sjoin, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
# save to file
sjoin.to_file('Documents/GIS DataBase/farmdams/geo_results/res_sjoin.shp')